**FIN 585R**  
**Diether**  
**Double Sorting**<br><br>

**Overview**

+ Goal: How to create double-sort portfolios<br><br>

+ Tool: Rely on a three-way groupby instead of a two-way grouupby.<br><br>

+ Also cover some odds and ends about working with the CRSP data.<br><br>


In [7]:
import numpy as np
import pandas as pd
from finance_byu.summarize import summary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 7.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


**Raw CRSP Data**

+ This is raw CRSP data in the feather format.<br><br>

+ The feather format can be read by pandas very quickly. It's a great format for large datasets.<br><br>

+ Raw CRSP data contains negative prices. CRSP reports a negative price if the price is a quote from a market market rather than an actual transaction prices.<br><br>

+ Typically researchers don't care about this distinction so just take the absolute value of price.<br><br>


In [8]:
df = pd.read_feather('12-mstk.ftr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4834887 entries, 0 to 4834886
Data columns (total 12 columns):
 #   Column     Dtype         
---  ------     -----         
 0   permno     int64         
 1   caldt      datetime64[ns]
 2   cusip      object        
 3   ticker     object        
 4   shrcd      int64         
 5   excd       int64         
 6   siccd      int64         
 7   prc        float64       
 8   ret        float64       
 9   vol        float64       
 10  shr        float64       
 11  cumfacshr  float64       
dtypes: datetime64[ns](1), float64(5), int64(4), object(2)
memory usage: 442.6+ MB


In [9]:
df[['prc','ret']].describe().round(3)

,prc,ret
count,4748212.000,4709657.000
mean,29.700,0.010
std,1575.692,0.170
min,-1925.000,-0.996
25%,2.812,-0.058
50%,12.570,0.000
75%,26.960,0.062
max,528921.000,24.000


In [10]:
df[['prc','ret']].quantile([0.05,0.1,0.15,0.20])

,prc,ret
0.05,-13.7500,-0.208333
0.10,-4.7500,-0.141484
0.15,-0.9375,-0.103489
0.20,1.2000,-0.077670


In [11]:
df['prc'] = df['prc'].abs()
df['me']  = df.eval("prc*shr/1000.0").where(df.eval("prc*shr > 1e-6"))

df[['prc','ret','me']].quantile([0.05,0.1,0.15,0.20])

,prc,ret,me
0.05,1.1250,-0.208333,3.165625
0.10,2.1250,-0.141484,6.003788
0.15,3.2899,-0.103489,9.516375
0.20,4.6200,-0.077670,13.992000


<br>

**Extension: Double Sorting Prep**

+ Sometimes you'll want to form portfolios based on two variables.<br><br>

+ We will form based on lagged market-cap and momentum.<br><br>

+ Need bins for both portfolio formation variables: momentum and market-cap.<br><br>

+ Let's use NYSE breakpoints for market-cap.<br><br>

+ We bin before splitting the sample so that the momentum breakpoints will be the same for the small and large-cap stratification.<br><br> 

+ Called independent double sorting. $\leftarrow$ Fama French (1992)<br><br>

+ Independent sorts make the comparisons across portfolio grouping more useful because the variation in momentum will be roughly the same across the portfolio groupings.<br><br>

In [12]:
df['prclag'] = df.groupby('permno')['prc'].shift(1)
df['melag'] = df.groupby('permno')['me'].shift(1)

df['logret'] = df.eval("log(1+ret)")
df['mom'] = df.groupby('permno')['logret'].rolling(11).sum().reset_index(drop=True)
df['mom'] = df.groupby('permno')['mom'].shift(2)

**NYSE Breakpoint Function**

In [13]:
# Only use market cap for NYSE (exchange code 1), bring it through entire dataframe, applied to everything
def nyse_qcut(x,bp=[0.3,0.7]):
    bins = x.query("excd == 1")['melag'].quantile(bp).searchsorted(x['melag'])
    return pd.DataFrame(bins,index=x.index)

In [14]:
df = df.query("mom == mom and melag == melag").reset_index(drop=True)

# Create bins for momentum separately from market cap
df['bins'] = df.groupby('caldt')['mom'].transform(pd.qcut,5,labels=False)

df['mebins'] = df.groupby('caldt')[['excd','melag']].apply(nyse_qcut)
df.head()

/var/folders/73/s_z37x2x4gv93j_kb8744gmh0000gn/T/ipykernel_13342/894084970.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['mebins'] = df.groupby('caldt')[['excd','melag']].apply(nyse_qcut)


,permno,caldt,cusip,ticker,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr,me,prclag,melag,logret,mom,bins,mebins
0,10000,1987-02-27,68391610,OMFGA,10,3,3990,0.406250,0.000000,392.0,3893.0,1.0,1.581531,0.406250,1.581531,0.000000,-2.138282,0,0
1,10000,1987-03-31,68391610,OMFGA,10,3,3990,0.250000,-0.384615,1111.0,3893.0,1.0,0.973250,0.406250,1.581531,-0.485508,-2.079442,0,0
2,10000,1987-04-30,68391610,OMFGA,10,3,3990,0.234375,-0.062500,998.0,3893.0,1.0,0.912422,0.250000,0.973250,-0.064539,-2.390878,0,0
3,10000,1987-05-29,68391610,OMFGA,10,3,3990,0.218750,-0.066667,815.0,3893.0,1.0,0.851594,0.234375,0.912422,-0.068993,-2.772589,0,0
4,10001,1987-02-27,39040610,GFGC,11,3,4920,6.250000,-0.074074,365.0,991.0,3.0,6.193750,6.750000,6.689250,-0.076961,0.196691,3,0


In [15]:
df = df.query("prclag >= 5").reset_index(drop=True)

In [16]:
port = df.groupby(['caldt','mebins','bins'])['ret'].mean()*100
port

caldt       mebins  bins
1927-01-31  0       0       -2.346467
                    1        5.537537
                    2        1.438325
                    3        4.562193
                    4       -2.419967
                              ...    
2022-09-30  2       0      -11.377753
                    1      -10.526091
                    2       -9.165679
                    3       -7.932932
                    4       -8.255344
Name: ret, Length: 17235, dtype: float64

In [17]:
port = port.unstack(level='bins')
port

bins                       0          1          2         3         4
caldt      mebins                                                     
1927-01-31 0       -2.346467   5.537537   1.438325  4.562193 -2.419967
           1       -2.878309   4.596625   1.609509  1.527852  1.424397
           2       -2.710620   1.405721  -0.434628  0.675446  0.362422
1927-02-28 0        4.232851   8.251475   7.414934  8.696680  2.099565
           1        8.273086   8.434622   5.105620  6.142898  6.811465
...                      ...        ...        ...       ...       ...
2022-08-31 1       -1.241922  -3.301169  -2.625694 -2.976684  0.063068
           2       -3.589977  -4.624896  -4.127457 -3.065706 -0.360051
2022-09-30 0      -12.814393 -10.800895  -7.970728 -4.864690 -4.847599
           1      -10.672518 -10.895457 -10.124792 -7.231977 -8.514846
           2      -11.377753 -10.526091  -9.165679 -7.932932 -8.255344

[3447 rows x 5 columns]

In [18]:
port.query("mebins == 0")

,bins,0,1,2,3,4
caldt,mebins,,,,,
1927-01-31,0,-2.346467,5.537537,1.438325,4.562193,-2.419967
1927-02-28,0,4.232851,8.251475,7.414934,8.696680,2.099565
1927-03-31,0,-2.591414,-1.201856,-3.037397,1.242627,-5.165859
1927-04-30,0,0.013754,0.285931,-2.749103,-0.025778,5.651458
1927-05-31,0,3.225591,12.713602,7.468712,10.494362,13.130511
...,...,...,...,...,...,...
2022-05-31,0,-4.668029,-0.823745,0.373583,0.071314,1.281933
2022-06-30,0,-2.101991,-7.171661,-6.136531,-3.881789,-7.594666
2022-07-29,0,7.834900,6.467267,5.058431,3.592868,5.063478


In [19]:
port = df.groupby(['caldt','mebins','bins'])['ret'].mean()*100
port = port.unstack(level=['mebins','bins'])
port

mebins              0                                                     1  \
bins                0          1         2          3          4          0   
caldt                                                                         
1927-01-31  -2.346467   5.537537  1.438325   4.562193  -2.419967  -2.878309   
1927-02-28   4.232851   8.251475  7.414934   8.696680   2.099565   8.273086   
1927-03-31  -2.591414  -1.201856 -3.037397   1.242627  -5.165859  -5.281953   
1927-04-30   0.013754   0.285931 -2.749103  -0.025778   5.651458  -2.573905   
1927-05-31   3.225591  12.713602  7.468712  10.494362  13.130511   3.109863   
...               ...        ...       ...        ...        ...        ...   
2022-05-31  -4.668029  -0.823745  0.373583   0.071314   1.281933  -3.214701   
2022-06-30  -2.101991  -7.171661 -6.136531  -3.881789  -7.594666  -6.680532   
2022-07-29   7.834900   6.467267  5.058431   3.592868   5.063478   8.966141   
2022-08-31  -0.960002  -2.151457 -2.879540  -1.670251   1.004125  -1.241922   
2022-09-30 -12.814393 -10.800895 -7.970728  -4.864690  -4.847599 -10.672518   

mebins                                                         2             \
bins                1          2         3          4          0          1   
caldt                                                                         
1927-01-31   4.596625   1.609509  1.527852   1.424397  -2.710620   1.405721   
1927-02-28   8.434622   5.105620  6.142898   6.811465   8.539571   5.733818   
1927-03-31  -3.606550  -2.123955 -1.221787  -0.055739  -5.990696  -6.037105   
1927-04-30  -0.652915   0.973914 -0.608256   3.159854  -8.523355  -1.103918   
1927-05-31   3.573679  11.184516  7.029676   8.779667   2.701414   2.114511   
...               ...        ...       ...        ...        ...        ...   
2022-05-31  -0.197945   0.373730  0.559953   2.057081  -7.077594   0.639661   
2022-06-30  -8.621817  -7.441312 -5.986213 -10.083759 -10.136220  -9.983260   
2022-07-29   9.303212   8.732595  7.031091   8.451605  10.136255  10.030570   
2022-08-31  -3.301169  -2.625694 -2.976684   0.063068  -3.589977  -4.624896   
2022-09-30 -10.895457 -10.124792 -7.231977  -8.514846 -11.377753 -10.526091   

mebins                                    
bins               2         3         4  
caldt                                     
1927-01-31 -0.434628  0.675446  0.362422  
1927-02-28  3.584392  3.502078  4.004880  
1927-03-31 -1.090184  1.175603  1.152143  
1927-04-30  0.443095  1.089239  1.787120  
1927-05-31  5.261142  7.583569  7.696873  
...              ...       ...       ...  
2022-05-31  0.420024  0.013716  0.986245  
2022-06-30 -8.243138 -7.111505 -8.707624  
2022-07-29  9.239871  7.841977  6.776446  
2022-08-31 -4.127457 -3.065706 -0.360051  
2022-09-30 -9.165679 -7.932932 -8.255344  

[1149 rows x 15 columns]

In [20]:
summary(port).loc[['mean','std','tstat']].round(3)

mebins      0                                  1                              \
bins        0      1      2      3      4      0      1      2      3      4   
mean    0.436  0.933  1.191  1.422  1.714  0.387  0.810  1.043  1.195  1.538   
std     9.019  7.450  6.849  7.131  7.814  9.217  7.214  6.407  5.963  6.856   
tstat   1.640  4.247  5.896  6.758  7.435  1.423  3.806  5.516  6.794  7.605   

mebins      2                              
bins        0      1      2      3      4  
mean    0.439  0.753  0.888  1.068  1.348  
std     9.107  6.857  5.886  5.504  6.072  
tstat   1.635  3.723  5.111  6.578  7.526